# Groq Tutorial

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
)

In [2]:
response = llm.invoke("the first person to step in moon was ...")
print(response.content)

The first person to step on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/lead-software-engineer-itc/job/R-51405")
page_data = loader.load().pop().page_content
page_data

USER_AGENT environment variable not set, consider setting it to identify your requests.


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLead Software Engineer, ITC\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\nOpen Virtual Assistant\n\n\n\n\n\n\n\n\n\n\nHome\n\n\nCareer Areas\n\n\nTotal Rewards\n\n\nLife@Nike\n\n\nPurpose\n\n\n\n\n\n\n\n\n\n\nLanguage\n\n\n\n\n\nSelect a Language\n\n  Deutsch  \n  English  \n  Español (España)  \n  Español (América Latina)  \n  Français  \n  Italiano  \n  Nederlands  \n  Polski  \n  Tiếng Việt  \n  Türkçe  \n  简体中文  \n  繁體中文  \n  עִברִית  \n  한국어  \n  日本語  \n\n\n\n\n\n\n\n\nCareers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nClose Menu\n\n\n\n\n\n\n\nCareers\n\n\n\n\n\n\nChat\n\n\n\n\n\n\n                                Home\n                            \n\n\n\n                                Career Areas\n                            \n\n\n\n                                Total Rewards\n                            \n\n\n\n                                Life@Nike\n                         

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT From website
    {page_data}
    ### INSTRUCTION:
    Your scraped text is from career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing following keys:
    'role', 'experience', 'description' and 'skills'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
print(type(res.content))


```json
{
  "role": "Lead Software Engineer, ITC",
  "experience": "7-10 years",
  "description": "We are seeking a ambitious Lead Full Stack Software Engineer to join our dynamic team. You’ll be responsible for developing and maintaining both front-end and back-end applications, ensuring seamless and scalable experiences for users.",
  "skills": [
    "UI Programming languages like React or Vue Js",
    "Server-side programming languages such as Node.js, Python",
    "Serverless architectures",
    "CI/CD pipeline",
    "AWS environments",
    "NoSQL databases like Dynamo DB",
    "RESTful APIs",
    "Cloud Technologies",
    "Application security concepts",
    "Version Control with Git, GitHub",
    "Testing Frameworks",
    "Problem-Solving",
    "Collaboration"
  ]
}
```
<class 'str'>


In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer, ITC',
 'experience': '7-10 years',
 'description': 'We are seeking a ambitious Lead Full Stack Software Engineer to join our dynamic team. You’ll be responsible for developing and maintaining both front-end and back-end applications, ensuring seamless and scalable experiences for users.',
 'skills': ['UI Programming languages like React or Vue Js',
  'Server-side programming languages such as Node.js, Python',
  'Designing and building serverless architectures',
  'Building end to end CI/CD pipeline',
  'NoSQL databases like Dynamo DB',
  'RESTful APIs and web services',
  'Cloud platforms preferably AWS',
  'Git, GitHub, or similar version control tools',
  'Unit testing, integration testing, and test-driven development (TDD)']}

In [9]:
type(json_res)

dict

In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectore_store')
collection = client.get_or_create_collection(name="portfolios")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=row["Techstack"],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )

In [12]:
links = collection.query(query_texts=["Experience in Python", "Experience in React"], n_results=2).get("metadatas")
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [13]:
jobs = json_res
jobs["skills"]

['UI Programming languages like React or Vue Js',
 'Server-side programming languages such as Node.js, Python',
 'Designing and building serverless architectures',
 'Building end to end CI/CD pipeline',
 'NoSQL databases like Dynamo DB',
 'RESTful APIs and web services',
 'Cloud platforms preferably AWS',
 'Git, GitHub, or similar version control tools',
 'Unit testing, integration testing, and test-driven development (TDD)']

In [20]:
prompt_email = PromptTemplate.from_template(
        """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are **Ravi Sharma**, a Business Consultant at **Techspire Solutions** — a forward-thinking tech consulting firm focused on helping companies build smarter, faster, and more scalable digital systems.
    Techspire specializes in delivering tailored AI and software solutions that simplify operations, accelerate product development, and cut costs — whether it's through automation, platform engineering, or custom development.
    Your task is to write a **cold email** to a potential client about the role described above. The email should:

    - Start with a **subject line** that clearly states the offer or value
    - Open with a friendly paragraph relating to their hiring or project needs
    - Casually explain how Techspire has solved similar challenges
    - Add the most relevant ones from the following links to show our portfolio {link_list}
    - Highlight 2–3 relevant past projects or portfolios using bullet points
    - Be persuasive without being salesy — think: “we get your world, and we can help”
    - End with a soft CTA like suggesting a quick call or sending case studies
    - Keep the tone friendly, confident, and focused on the client’s benefit
    - Format the mail in a **business-casual style**, using bullet points and short paragraphs

    You're not writing to “sell” — you're starting a conversation and showing how Techspire can add real value.
    You are **Ravi Sharma**, Business Consultant at **Techspire Solutions**.

    ### EMAIL (NO PREAMBLE):
    """
)


In [21]:
type(str(jobs))

str

In [22]:
chain_email = prompt_email | llm
res = chain_email.invoke(
    {"job_description": str(jobs), "link_list": links}
)
print(res.content)

Subject: Expert Software Engineering Solutions for Your Business

Hi there,

I hope this email finds you well. As a hiring manager or project lead, you're likely no stranger to the challenges of finding the right talent to drive your digital initiatives forward. With the demand for skilled software engineers on the rise, it can be tough to find the perfect fit for your team. That's where Techspire Solutions comes in – we specialize in delivering tailored software solutions that simplify operations, accelerate product development, and cut costs.

We've helped numerous clients overcome similar challenges by providing them with top-notch software engineering expertise. Our team has a proven track record of developing and maintaining scalable, efficient, and user-friendly applications. You can check out some of our relevant work here: 
https://example.com/react-portfolio, 
https://example.com/python-portfolio. 

Some of our notable projects include:
* Building a serverless architecture for